In [1]:
# import libraries
import re
import pandas as pd
import csv
import numpy as np

In [2]:
# load waltz data
waltz_data = pd.read_csv('Waltz_and_AAIndex1')
waltz_data

C:\Users\Freddie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Classification,Sequence,pos0,pos1,pos2,pos3,pos4,pos5,pos0_orth,pos1_orth,...,pos2_KARS160121,pos3_KARS160121,pos4_KARS160121,pos5_KARS160121,pos0_KARS160122,pos1_KARS160122,pos2_KARS160122,pos3_KARS160122,pos4_KARS160122,pos5_KARS160122
0,non-amyloid,AAAQAA,A,A,A,Q,A,A,1000000000000000000000,1000000000000000000000,...,6.000,10.500,6.000,6.000,0.000,0.000,0.000,1.849,0.000,0.000
1,non-amyloid,AAELRN,A,A,E,L,R,N,1000000000000000000000,1000000000000000000000,...,10.667,9.600,10.667,10.000,0.000,0.000,1.822,3.113,4.200,3.000
2,non-amyloid,AAIDWF,A,A,I,D,W,F,1000000000000000000000,1000000000000000000000,...,9.600,10.400,12.750,12.000,0.000,0.000,3.373,2.969,2.044,2.026
3,non-amyloid,AAIGWG,A,A,I,G,W,G,1000000000000000000000,1000000000000000000000,...,9.600,3.500,12.750,3.500,0.000,0.000,3.373,0.000,2.044,0.000
4,non-amyloid,AALQSS,A,A,L,Q,S,S,1000000000000000000000,1000000000000000000000,...,9.600,10.500,8.667,8.667,0.000,0.000,3.113,1.849,6.000,6.000
5,non-amyloid,AAPKPK,A,A,P,K,P,K,1000000000000000000000,1000000000000000000000,...,12.000,10.167,12.000,10.167,0.000,0.000,12.000,1.372,12.000,1.372
6,non-amyloid,AAQAAL,A,A,Q,A,A,L,1000000000000000000000,1000000000000000000000,...,10.500,6.000,6.000,9.600,0.000,0.000,1.849,0.000,0.000,3.113
7,non-amyloid,AARRFF,A,A,R,R,F,F,1000000000000000000000,1000000000000000000000,...,10.667,10.667,12.000,12.000,0.000,0.000,4.200,4.200,2.026,2.026
8,non-amyloid,AAVDQT,A,A,V,D,Q,T,1000000000000000000000,1000000000000000000000,...,9.000,10.400,10.500,9.000,0.000,0.000,6.000,2.969,1.849,6.000
9,non-amyloid,ACGVIG,A,C,G,V,I,G,1000000000000000000000,0010000000000000000000,...,3.500,9.000,9.600,3.500,0.000,6.000,0.000,6.000,3.373,0.000


In [3]:
# define a function to count matching amino acids in same position for two peptide sequences
def similarity(string1, string2):
    overlap = 0
    # make sure string1 is at least shorter than string2
    if (len(string2) < len(string1)):
        return similarity(string2, string1)
    else:
        for i in range(0, len(string1)):
            if (string1[i] == string2[i]):
                overlap += 1
    return overlap

In [31]:
# set the threshold maximum similarity
threshold = 3

# create an empty list, to which 'non-similar' sequences will be added
non_similar_list = []

# loop through all of the sequences in the waltz_data
for sequence1 in waltz_data['Sequence']:
    # create a list of the similarities between sequence1 and every element in the non_similar_list
    list_matches = [similarity(sequence1, y) for y in non_similar_list]
    # if no sequence has similarity greater than the threshold, then add sequence1 to the non_similar_list
    if (max(list_matches, default = 0) <= threshold):
        non_similar_list.append(sequence1)

non_similar_list

['AAAQAA',
 'AAELRN',
 'AAIDWF',
 'AALQSS',
 'AAPKPK',
 'AAQAAL',
 'AARRFF',
 'AAVDQT',
 'ACGVIG',
 'ADVGQG',
 'ADVSIE',
 'AEKLFD',
 'AEMEYL',
 'AENGKS',
 'AESDKK',
 'AETKEP',
 'AEVLAL',
 'AFSINP',
 'AGFQPQ',
 'AGGYYQ',
 'AGSGSG',
 'AGVNYF',
 'AIDWFD',
 'AIGWGS',
 'AILSST',
 'AINKIQ',
 'AISNNN',
 'AIVAGD',
 'ALALQT',
 'ALEEYT',
 'ALQSSW',
 'ALQTDA',
 'AMMAAA',
 'AMNITN',
 'ANATKK',
 'ANNNSN',
 'APKPKK',
 'AQAALQ',
 'AQFIIS',
 'AQPAGG',
 'ASNAGS',
 'ASQQNQ',
 'ASSSNY',
 'ATAHQY',
 'ATKKVD',
 'ATLDQW',
 'ATTRTV',
 'ATVSFD',
 'AVDQTA',
 'AVTTGV',
 'AYNAQA',
 'CGVIGI',
 'CPSGNN',
 'DADLYL',
 'DAGYQQ',
 'DCVNIT',
 'DDSLFF',
 'DECFFF',
 'DENNTF',
 'DETVIV',
 'DFQKQQ',
 'DHCIIG',
 'DIEVDL',
 'DILTYT',
 'DKKEEE',
 'DLLKNG',
 'DLTITQ',
 'DLTQRG',
 'DNNTIF',
 'DQTASN',
 'DQWNGK',
 'DSNQGN',
 'DSVISL',
 'DTKPAE',
 'DTVIIE',
 'DVGQGS',
 'DVNRNG',
 'DVSIED',
 'EALYLV',
 'EATVSF',
 'ECFFFE',
 'EDSVIS',
 'EECLFL',
 'EEEKSA',
 'EEKKEE',
 'EEKPVQ',
 'EEPVKK',
 'EGVLYV',
 'EHSDLS',
 'EKDEYA',
 'EKNLYL',

In [37]:
waltz_data = waltz_data[waltz_data['Sequence'].isin(non_similar_list)]
waltz_data.describe()

,pos0_ANDN920101,pos1_ANDN920101,pos2_ANDN920101,pos3_ANDN920101,pos4_ANDN920101,pos5_ANDN920101,pos0_ARGP820101,pos1_ARGP820101,pos2_ARGP820101,pos3_ARGP820101,...,pos2_KARS160121,pos3_KARS160121,pos4_KARS160121,pos5_KARS160121,pos0_KARS160122,pos1_KARS160122,pos2_KARS160122,pos3_KARS160122,pos4_KARS160122,pos5_KARS160122
count,652.000000,652.000000,652.000000,652.000000,652.000000,651.000000,652.000000,652.000000,652.000000,652.000000,...,652.000000,652.000000,652.000000,651.000000,652.000000,652.000000,652.000000,652.000000,652.000000,651.000000
mean,4.374202,4.348834,4.386120,4.368098,4.368451,4.378387,0.761012,0.819172,0.844248,0.858635,...,9.511534,9.451689,9.568155,9.442140,2.903454,3.059370,3.067650,3.156015,3.032187,3.137223
std,0.264244,0.257499,0.274904,0.267371,0.274880,0.251537,0.774885,0.791067,0.823666,0.829791,...,2.409708,2.266633,2.266948,2.410149,2.466355,2.476509,2.437063,2.444277,2.317429,2.528573
min,3.950000,3.950000,3.950000,3.950000,3.950000,3.950000,0.000000,0.000000,0.000000,0.000000,...,3.500000,3.500000,3.500000,3.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.170000,4.170000,4.170000,4.170000,4.170000,4.290000,0.060000,0.060000,0.060000,0.060000,...,9.000000,9.000000,9.000000,9.000000,1.599000,1.599000,1.822000,1.822000,1.605000,1.822000
50%,4.370000,4.355000,4.370000,4.360000,4.370000,4.370000,0.535000,0.610000,0.610000,0.610000,...,10.000000,10.000000,10.000000,10.000000,2.656000,2.969000,2.656000,3.000000,2.969000,2.656000
75%,4.600000,4.520000,4.660000,4.600000,4.600000,4.560000,1.320000,1.530000,1.530000,1.530000,...,10.667000,10.500000,10.541750,10.667000,3.373000,4.200000,3.373000,3.373000,3.373000,6.000000
max,4.760000,4.760000,4.760000,4.760000,4.760000,4.760000,2.650000,2.650000,2.650000,2.650000,...,13.600000,13.600000,13.600000,13.600000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
